# Imorting The Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import tensorflow as tf                       
import numpy as np                           
import matplotlib.pyplot as plt               
%matplotlib inline

# LOAD THE DATA

In [ ]:
from tensorflow.keras.datasets.mnist import load_data #fetch the data from keras

In [ ]:
(X_train, y_train) , (X_test, y_test) = load_data()  #you can think this as train_test_split

In [ ]:
print("Train size:", len(X_train))     
print("Test size:", len(X_test))  

In [ ]:
X_train[24]  #Let's test some images

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(X_train[24], cmap= plt.cm.binary); #visualize the matrix above

In [ ]:
num_rows, num_cols = 3, 6                                       # Datadan 3 tane row, 6 tane column sectik.
f, ax = plt.subplots(3, 6, figsize=(12,5),
                     gridspec_kw={'wspace':0.04, 'hspace':0.03}, 
                     squeeze=True)

for r in range(num_rows):
    for c in range(num_cols):
      
        image_index = r * 5 + c
        ax[r,c].axis("off")
        ax[r,c].imshow( X_train[image_index], cmap='gray')
plt.show()
plt.close()

Lets look some matrix values in order to see if data is scaled or not

In [ ]:
X_train[19].min() #would be 0-1 if it's scaled

In [ ]:
X_train[19].max() #would be 0-1 if it's scaled

## SCALING DATA

In [ ]:
#Let's scale
X_train = X_train.astype('float32') #Float conversion is used to prevent rounding numbers to integers
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
X_train[23].min() #now it's scaled

In [ ]:
X_train[23].max()

In [ ]:
X_train = X_train.reshape(X_train.shape[0],28, 28, 1) 
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
X_train.shape, X_test.shape #now it's converted from matrix to tensor

In [ ]:
X_train[245].shape 

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
Y_train = to_categorical(y_train, 10) 
Y_test = to_categorical(y_test, 10)
Y_test.shape

# BUILD MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout,Input

In [ ]:
#Defien the input shape
input_shape=X_train[123].shape
input_shape

In [ ]:
model = Sequential()
#model.add(Input(shape=input_shape)) #It helps model to compile better when u add the input as a "LAYER"
#CNN
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(28, 28, 1))) # if we use padding='valid' we would have lost shape of the images
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Flatten())
#ANN

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# OUTPUT LAYER
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# MODEL SUMMARY

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", 
                           min_delta=0, #val_loss value between two epochs
                           mode="auto",
                           restore_best_weights = True, #in case if model misses the best_weights
                           verbose=1,
                           patience=50)

In [ ]:
model.fit(X_train,
          Y_train, #do not forget to use categorical form of Y, if you use y_train you will get dimension error!!!
          validation_split=0.1,
          batch_size=512,
          epochs=100,
          verbose=1,
          callbacks=[early_stop])

In [ ]:
model.history.history

In [ ]:
summary = pd.DataFrame(model.history.history)
summary.head(50)[10:20:1]

In [ ]:
model.evaluate(X_test,Y_test)[1] #to see test accuracy

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.accuracy)        
plt.plot(summary.val_accuracy)
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend('labels');

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.accuracy)        
plt.plot(summary.val_accuracy)
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend('labels');

plt.ylim(0.95,1) #in order to zoom in

#  PREDICTION

In [ ]:
preds = model.predict(X_test)
preds

In [ ]:
preds[0] #We have probabilities of each class we have to select the highest prob value

In [ ]:
predictions= np.argmax(preds, axis=1)

In [ ]:
predictions #now we can see which label model predicted

In [ ]:
print(predictions[:100])

# EVALUATION

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(confusion_matrix(y_test, predictions))

# PREDICTION OF TEST VALUES

In [ ]:
number_test = X_test[24]
plt.imshow(number_test); 

In [ ]:
np.argmax(model.predict(number_test.reshape(1,28,28,1)), axis=1)